In [1]:
## Auto reload
%load_ext autoreload
%autoreload 2

from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
import thunder
import pickle
import torch
import os
        
torch.set_float32_matmul_precision('medium')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float32
print(f"Using dtype: {dtype}")

Using device: cuda
Using dtype: torch.bfloat16


In [10]:
from private.hyper_modules import RecurrentHyperLayer #, ParallelScanRecurrentLayer

layer = RecurrentHyperLayer(
    hidden_size=512,
    hyper_features=16,
    intermediate_size=2048,
    dora_rank=8,
    ema_steps = 32,
).to(device=device, dtype=dtype)

layer=torch.compile(layer)

total_params = sum(p.numel() for p in layer.parameters()) * 8
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 42,777,344


In [11]:
x = torch.rand((64,256,512), device=device, dtype=torch.bfloat16)

In [19]:
y=layer(x)

In [13]:
y.shape

torch.Size([64, 256, 512])